In [1]:
import girder_client 
%load_ext autoreload
%autoreload 2
#import adrcSchemaHelpers as hlprs
import metadata_pipeline as mph 
import brain_region_maps as brm

#gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [2]:
NeuroPathDemoImages_Collection='638e2da11f75016b81fda12f'

In [5]:
mph.populateMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputFailed=True)

{'63ab24647b54d4a0848fba0c': {'name': 'E39-44_TAU.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_TAU.ndpi failed to parse'}}},
 '63ab24647b54d4a0848fba0f': {'name': 'E39-44_BLANK.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_BLANK.ndpi failed to parse'}}},
 '63ab24647b54d4a0848fba12': {'name': 'E39-44_AB.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_AB.ndpi failed to parse'}}},
 '63ab24657b54d4a0848fba14': {'name': 'OS89-44_Ab.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name OS89-44_Ab.ndpi failed to parse'}}},
 '63ab24667b54d4a0848fba1a': {'name': 'PSTVE_CNTRL_FOR_OS89-44_AB.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name PSTVE_CNTRL_FOR_OS89-44_AB.ndpi failed to parse'}}},
 '63ab24667b54d4a0848fba1d': {'name': 'E39-44_SYN.ndpi',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_SYN.ndpi failed to parse'}}},
 '63ab13a97b54d4a0848f6d73': {'name': '1028860.svs',
  'meta': {'npWorking': {

In [7]:
mph.auditMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputRecords=True)

In [59]:
## Need to do this on a per case basis
needsBrainRegionMap = []
slideSummaryData = {}

for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
    year = yrf['name']
    if year.isdigit():
        slideSummaryData[year] = {} ## Going to create a stats dictionary ordered by year/pt
        
        for pt in gc.listFolder(yrf['_id']):
            slideSummaryData[year][pt['name']] = {}
            
            if pt['name'] not in brm.MAP:
                print(year,pt['name'],"Needs Brain Region Map")
                slideSummaryData[year][pt['name']] = {"status": "incomplete", "reason": "needs brain region map"}
        
            else:

                ptItemData = mph.getFolderContents(gc,pt['_id']) ### Get all of the items cfor selected year
        
                stats = getPtSlideDataSummary(ptItemData,debug=True)
                slideSummaryData[year][pt['name']] = stats

1989 E39-44 Needs Brain Region Map
1994 OS94-54 Needs Brain Region Map
1996 E96-33 Needs Brain Region Map
1997 E97-53 Needs Brain Region Map
1997 E97-60 Needs Brain Region Map
1997 E97-61 Needs Brain Region Map
1999 E99-04 Needs Brain Region Map
1999 E99-07 Needs Brain Region Map
1999 E99-26 Needs Brain Region Map
1999 E99-30 Needs Brain Region Map
1999 OS99-22 Needs Brain Region Map
2000 E00-06 Needs Brain Region Map
2000 E00-10 Needs Brain Region Map
2000 E00-11 Needs Brain Region Map
2000 E00-12 Needs Brain Region Map
2000 E00-14 Needs Brain Region Map
2000 E00-18 Needs Brain Region Map
2000 E00-19 Needs Brain Region Map
2000 E00-21 Needs Brain Region Map
2000 E00-28 Needs Brain Region Map
2000 E00-32 Needs Brain Region Map
2000 E00-40 Needs Brain Region Map
2000 OS00-40 Needs Brain Region Map
2000 OS00-46 Needs Brain Region Map
2000 OS00-47 Needs Brain Region Map
2000 OS00-48 Needs Brain Region Map
2001 E01-06 Needs Brain Region Map
2001 E01-33 Needs Brain Region Map
2001 E01-70 Ne

Valid:86,Controls:0,NotValid:46
Valid:78,Controls:0,NotValid:42
Valid:0,Controls:0,NotValid:77
2010 E10-66 Needs Brain Region Map
2010 E10-77 Needs Brain Region Map
Valid:180,Controls:2,NotValid:40
2010 E10-83 Needs Brain Region Map
Valid:78,Controls:0,NotValid:11
2010 E10-95 Needs Brain Region Map
2010 OS082310 Needs Brain Region Map
2011 E11-07 Needs Brain Region Map
2011 E11-108 Needs Brain Region Map
2011 E11-11 Needs Brain Region Map
2011 E11-111 Needs Brain Region Map
Valid:78,Controls:0,NotValid:46
2011 E11-121 Needs Brain Region Map
Valid:106,Controls:0,NotValid:16
Valid:109,Controls:0,NotValid:63
Valid:98,Controls:0,NotValid:9
Valid:76,Controls:1,NotValid:54
Valid:96,Controls:2,NotValid:31
2011 E11-165 Needs Brain Region Map
Valid:86,Controls:1,NotValid:15
2011 E11-32 Needs Brain Region Map
2011 E11-33 Needs Brain Region Map
Valid:98,Controls:0,NotValid:3
2011 E11-55 Needs Brain Region Map
Valid:89,Controls:0,NotValid:14
Valid:81,Controls:0,NotValid:33
2011 E11-62 Needs Brain 

Valid:147,Controls:8,NotValid:0
Valid:132,Controls:4,NotValid:0
Valid:128,Controls:0,NotValid:0
Valid:127,Controls:7,NotValid:0
Valid:75,Controls:8,NotValid:2
Valid:110,Controls:6,NotValid:0
Valid:99,Controls:4,NotValid:0
2020 OS-201117 Needs Brain Region Map
Valid:0,Controls:5,NotValid:80
Valid:0,Controls:2,NotValid:111
Valid:0,Controls:2,NotValid:80
Valid:0,Controls:6,NotValid:128
Valid:0,Controls:0,NotValid:76
Valid:0,Controls:1,NotValid:78
Valid:84,Controls:6,NotValid:0
Valid:0,Controls:4,NotValid:85
Valid:0,Controls:1,NotValid:85
Valid:86,Controls:1,NotValid:0
Valid:75,Controls:2,NotValid:0
Valid:68,Controls:0,NotValid:0
Valid:79,Controls:4,NotValid:0
Valid:0,Controls:5,NotValid:74
Valid:0,Controls:5,NotValid:87
Valid:0,Controls:3,NotValid:106
Valid:0,Controls:5,NotValid:71
2021 OS-211103 Needs Brain Region Map
2022 E22-02 Needs Brain Region Map
2022 E22-03 Needs Brain Region Map
2022 E22-15 Needs Brain Region Map
2022 E22-39 Needs Brain Region Map
2022 E22-48 Needs Brain Region M

In [51]:
mph.auditMetadata(gc,folderID='638e81331f75016b81febf39',outputRecords=True)

In [66]:
folder2020 = '638e81331f75016b81febf39'

itemData2020 = mph.getFolderContents(gc,folder2020)

In [69]:
allNpWorking = []

for itm in itemData2020:
    if 'meta' in itm and 'npWorking' in itm['meta']:
        allNpWorking.append(itm)

In [74]:
[(x['meta']['npWorking'],x['name']) for x in allNpWorking]

[({'stainID': "'E' is not one of ['pTDP', 'HE', 'aBeta', 'Ubiq', 'Tau', 'Biels', 'Syn', 'p62', 'LFB', 'FUS', 'TDP-43', 'GFAP']"},
  'E20-11_9HE.svs'),
 ({'Metadata Error': 'File name failed to parse'}, '1029075.svs'),
 ({'Metadata Error': 'File name failed to parse'}, '1029076.svs'),
 ({'stainID': "'AU' is not one of ['pTDP', 'HE', 'aBeta', 'Ubiq', 'Tau', 'Biels', 'Syn', 'p62', 'LFB', 'FUS', 'TDP-43', 'GFAP']"},
  'E20-63_7TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'Tau.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_10_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_14_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_5_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_6_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_7_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse'}, 'OS-201117_8_TAU.svs'),
 ({'Metadata Error': 'File name failed to parse

In [62]:
for pt in slideSummaryData['2017'].keys():
    s=  slideSummaryData['2017'][pt]
    if 'valid' in s:
        print('2017',pt,s['valid'],s['notValid'],s['controlSlides'])
    

2017 E17-05 96 2 4
2017 E17-12 110 1 4
2017 E17-128 54 2 2
2017 E17-129 44 2 2
2017 E17-138 72 77 17
2017 E17-140 55 0 1
2017 E17-141 70 2 4
2017 E17-143 118 0 5
2017 E17-147 84 0 3
2017 E17-150 83 0 3
2017 E17-152 95 1 1
2017 E17-17 94 4 6
2017 E17-21 103 0 5
2017 E17-22 89 4 3
2017 E17-32 74 0 0
2017 E17-36 92 1 5
2017 E17-39 91 73 5
2017 E17-40 83 0 0
2017 E17-47 76 0 0
2017 E17-53 79 10 4
2017 E17-54 76 83 6
2017 E17-56 70 0 3
2017 E17-57 86 0 3
2017 E17-71 70 0 6
2017 E17-88 68 3 0


In [65]:
for i in slideSummaryData['2017']['E17-39']['slidesToReview']:
    print(i)

{'_id': '638e9b831f75016b81fefd21', 'baseParentId': '638e2da11f75016b81fda12f', 'baseParentType': 'collection', 'created': '2022-12-06T01:31:47.948000+00:00', 'creatorId': '638e2ae7cd1d7fad17b9dd1f', 'description': '', 'folderId': '638e9b821f75016b81fefd1e', 'largeImage': {'fileId': '638e9b841f75016b81fefd22', 'sourceName': 'openslide'}, 'lowerName': 'e17-39_gfap_r7.svs', 'meta': {'npWorking': {'Metadata Error': 'File name failed to parse'}}, 'name': 'E17-39_GFAP_R7.svs', 'size': 883794573, 'updated': '2022-12-15T00:39:58.349000+00:00'}
{'_id': '638e9b8c1f75016b81fefd33', 'baseParentId': '638e2da11f75016b81fda12f', 'baseParentType': 'collection', 'created': '2022-12-06T01:31:56.084000+00:00', 'creatorId': '638e2ae7cd1d7fad17b9dd1f', 'description': '', 'folderId': '638e9b821f75016b81fefd1e', 'largeImage': {'fileId': '638e9b8c1f75016b81fefd34', 'sourceName': 'openslide'}, 'lowerName': 'e17-39_lfb_r-9.svs', 'meta': {'npWorking': {'Metadata Error': 'File name failed to parse'}}, 'name': 'E

In [ ]:
allItemData = mph.get

In [44]:
def getPtSlideDataSummary( slideSetMetaData,debug=True):
    ## Expects an array of all of the items below a folder, so for exampel  all of the slides for case E20-73
    ## This should then return the number of control slides found, number of files that are missing metadata
    ## completely, and also invalid metadata
    ctrlSlides = 0
    validatedSlides = 0
    notValid = 0
    slidesToReview = []
    
    for i in slideSetMetaData:
        if 'npSchema' not in i['meta']:
            notValid+=1
            slidesToReview.append(i)
        elif ('controlSlide' in i['meta']['npSchema']):
            ctrlSlides+=1
        else:
            validItem,meta = mph.validateMetadata(i['meta'])
            ### Now process errors from 
            if not validItem:
                notValid+=1
                slidesToReview.append(i)
            else:
                validatedSlides+=1
    if debug:
        print("Valid:%d,Controls:%d,NotValid:%d" % (validatedSlides,ctrlSlides,notValid))
        
        
    return {"valid":validatedSlides,"controlSlides": ctrlSlides, "notValid": notValid, "slidesToReview":slidesToReview}

In [ ]:
# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             print(ptId['name'])
#             hlprs.scanMetadata(gc,ptId,updateGirder=True)

In [ ]:
# ## Now also flatten the views so in the image organizer app I don't have to upload

# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             gc.addMetadataToFolder(ptId['_id'],{"isLinear":True, "caseViewFlag":1})

In [19]:
# ### Get a list of every patient ID... this is quick, it's just from the main folder name!!
# ptIdList = []

# for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     year = yrf['name']
#     if year.isdigit():
#         for ptf in gc.listFolder(yrf['_id']):
# #             print("\t",ptf['name'])
#             ptIdList.append(ptf['name'])